In [29]:
import numpy as np
import pandas as pd
import locale
import json
import re
import torch

from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util

In [2]:
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [3]:
# Dates to datetime format in news dataset
news_data = pd.read_json('/Users/alexanderknyshov/Desktop/LLM/Data/datasets/news_data_new.json', lines=True)
date = []
for i in range(len(news_data['publishedAt'])):
    date.append(pd.to_datetime(news_data['publishedAt'][i], dayfirst=True, errors='coerce'))
for i in range(len(news_data['publishedAt'])):
    news_data.loc[i, 'publishedAt'] = date[i]

In [4]:
# Joint recommendations dataset
rec1 = pd.read_json('/Users/alexanderknyshov/Desktop/LLM/Data/datasets/invest_recommendations.json')
rec2 = pd.read_json('/Users/alexanderknyshov/Desktop/LLM/Data/datasets/recommendations.json')

rec2 = rec2.rename(columns={'titles': 'title'})
rec2 = rec2.drop(columns=['hrefs'])

In [5]:
# Dates to datetime format in rec1 and rec2
date = []
for i in range(len(rec1['publishedAt'])):
    date.append(pd.to_datetime(rec1['publishedAt'][i], format='%d %B %Y, %H:%M'))
for i in range(len(rec1['publishedAt'])):
    rec1.loc[i, 'publishedAt'] = date[i]
    
date = []
for i in range(len(rec2['publishedAt'])):
    date.append(pd.to_datetime(rec2['publishedAt'][i], format='%d %B %Y', errors='coerce'))
for i in range(len(rec2['publishedAt'])):
    rec2.loc[i, 'publishedAt'] = date[i]

# Merging rec1 and rec2 to get final dataset
recommendations = pd.concat([rec1, rec2], ignore_index=True)

In [6]:
rec_news = []
for i in range(len(recommendations['publishedAt'])):
    date = recommendations['publishedAt'][i]
    start_date = date - pd.Timedelta(days=7)
    filtered_news = news_data[(news_data['publishedAt'] >= start_date) & (news_data['publishedAt'] < date)]['description']
    # filtered_news = str(filtered_news.sum())
    rec_news.append(filtered_news)

# recommendations['reason news'] = rec_news

In [19]:
lst = list(rec_news[0])

In [20]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")

/Users/alexanderknyshov/Desktop/LLM/Data/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.

In [30]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

Error while downloading from https://cdn-lfs.huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/eaa086f0ffee582aeb45b36e34cdd1fe2d6de2bef61f8a559a1bbc9bd955917b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1727448928&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNzQ0ODkyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zZW50ZW5jZS10cmFuc2Zvcm1lcnMvcGFyYXBocmFzZS1tdWx0aWxpbmd1YWwtTWluaUxNLUwxMi12Mi9lYWEwODZmMGZmZWU1ODJhZWI0NWIzNmUzNGNkZDFmZTJkNmRlMmJlZjYxZjhhNTU5YTFiYmM5YmQ5NTU5MTdiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=fzrEpXuYo9LrtzEBA6pU5ojxugFvO%7EVV%7EwGjF6K0NjQzk1qyzQ1rT0x6yPwsuV8wFOVrqCHSc70FjBJsi8EP2HQj%7EWPeAAA8IhKLiojCXlbslgytODBM413q9RLxaXHYmA4xNSf0iyexaW6lmHO6DGPBcUPksxZPtGIUPGF9Q03ntThzxqfUkMthIie7AeORulCeeXsb3cRCw2qhkjGvyAWe4BbRQRYq8bYq1OmVRYnHqPpQiiWccQDyugJsfeYIYqA3RMl28p5-syc%7ExCP1s8RhM

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/eaa086f0ffee582aeb45b36e34cdd1fe2d6de2bef61f8a559a1bbc9bd955917b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1727448928&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNzQ0ODkyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zZW50ZW5jZS10cmFuc2Zvcm1lcnMvcGFyYXBocmFzZS1tdWx0aWxpbmd1YWwtTWluaUxNLUwxMi12Mi9lYWEwODZmMGZmZWU1ODJhZWI0NWIzNmUzNGNkZDFmZTJkNmRlMmJlZjYxZjhhNTU5YTFiYmM5YmQ5NTU5MTdiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=fzrEpXuYo9LrtzEBA6pU5ojxugFvO~VV~wGjF6K0NjQzk1qyzQ1rT0x6yPwsuV8wFOVrqCHSc70FjBJsi8EP2HQj~WPeAAA8IhKLiojCXlbslgytODBM413q9RLxaXHYmA4xNSf0iyexaW6lmHO6DGPBcUPksxZPtGIUPGF9Q03ntThzxqfUkMthIie7AeORulCeeXsb3cRCw2qhkjGvyAWe4BbR

In [23]:
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Получаем вектор из [CLS] токена
    return outputs.last_hidden_state[:, 0, :]

# Ваши данные
news = lst
recommendation = recommendations['description'][0]
recommendation_embedding = get_sentence_embedding(recommendation)
cos = []

for i in range(len(news)):
    news_embedding = get_sentence_embedding(news[i])
    cosine_similarity = torch.nn.functional.cosine_similarity(news_embedding, recommendation_embedding)
    cos.append(cosine_similarity)
    print(f"Semantic similarity for news {i}: {cosine_similarity.item()}")

Semantic similarity for news 0: 0.8513872623443604
Semantic similarity for news 1: 0.8526464700698853
Semantic similarity for news 2: 0.8896920680999756
Semantic similarity for news 3: 0.8936219215393066
Semantic similarity for news 4: -0.054632704704999924
Semantic similarity for news 5: 0.8811293244361877
Semantic similarity for news 6: 0.8953263163566589
Semantic similarity for news 7: 0.8800795674324036
Semantic similarity for news 8: 0.8895751237869263
Semantic similarity for news 9: 0.9011843204498291
Semantic similarity for news 10: -0.054632704704999924
Semantic similarity for news 11: 0.8938632011413574
Semantic similarity for news 12: 0.8834102749824524
Semantic similarity for news 13: 0.888358473777771
Semantic similarity for news 14: 0.9128526449203491
Semantic similarity for news 15: 0.8960676193237305
Semantic similarity for news 16: -0.054632704704999924
Semantic similarity for news 17: 0.8634659647941589
Semantic similarity for news 18: 0.8648092150688171
Semantic simil

[tensor([0.8514]),
 tensor([0.8526]),
 tensor([0.8897]),
 tensor([0.8936]),
 tensor([-0.0546]),
 tensor([0.8811]),
 tensor([0.8953]),
 tensor([0.8801]),
 tensor([0.8896]),
 tensor([0.9012]),
 tensor([-0.0546]),
 tensor([0.8939]),
 tensor([0.8834]),
 tensor([0.8884]),
 tensor([0.9129]),
 tensor([0.8961]),
 tensor([-0.0546]),
 tensor([0.8635]),
 tensor([0.8648]),
 tensor([0.8813]),
 tensor([0.9139])]

In [25]:
cos.index(max(cos))

20

In [26]:
news[20]

'Идея, овладевшая массами, становится материальной силой, учили классики марксизма. С этой точки зрения идея конфискации российских валютных резервов весьма близка к воплощению: она стремительно покоряет евроатлантический политбомонд. Об этом свидетельствуют, в частности, недавние высказывания главы евродипломатии Жозепа Борреля. Может ли Россия спасти свои арестованные активы от экспроприации?'

In [27]:
recommendation

'Премьер-министр РФ Михаил Мишустин подписал постановление правительства о расширении механизма использования инфраструктурных облигаций.\nЧто нового в Постановлении Правительства по инфраструктурным облигациям\n\n\nТеперь можно будет использовать инфраструктурные облигации для финансирования проектов благоустройства городских и сельских территорий.\n\n\nМеханизм инфраструктурных облигаций будет распространяться на проекты со сроком реализации до 49 лет (раньше он был до 30 лет).\n\n\nОбязательства эмитента могут быть обеспечены госгарантией и независимыми гарантиями государственных корпораций развития.\n\n\nК эмитентам, реализующим проекты на Дальнем Востоке и в Арктике, будут снижены требования по достаточности собственных средств: с 20% до 10% от стоимости проекта.\xa0\n\n\nА что такое инфраструктурные облигации простыми словами?\nЭто понятие появилось в российском законодательстве совсем недавно — в 2020 году, хотя в мировой практике такие облигации используются уже давно и довольн